### Converting PASCAL VOC Format Annotations to YOLO compatible format


In [ ]:
import os
import xml.etree.ElementTree as ET


def convert_voc_to_yolo(voc_folder, yolo_folder, classes):
    for filename in os.listdir(voc_folder):
        if filename.endswith(".xml"):
            xml_path = os.path.join(voc_folder, filename)
            yolo_path = os.path.join(
                yolo_folder, os.path.splitext(filename)[0] + ".txt")

            with open(yolo_path, "w") as yolo_file:
                print(f"Processing {filename}")

                tree = ET.parse(xml_path)
                root = tree.getroot()

                width = int(root.find("size/width").text)
                height = int(root.find("size/height").text)

                print(f"Image Size: {width} x {height}")

                for obj in root.findall("object"):
                    class_name = obj.find("name").text
                    print(f"Class: {class_name}")

                    if class_name not in classes:
                        print(f"Skipping class: {class_name}")
                        continue

                    class_id = classes.index(class_name)

                    bbox = obj.find("bndbox")
                    xmin = float(bbox.find("xmin").text)
                    ymin = float(bbox.find("ymin").text)
                    xmax = float(bbox.find("xmax").text)
                    ymax = float(bbox.find("ymax").text)

                    x_center = (xmin + xmax) / (2 * width)
                    y_center = (ymin + ymax) / (2 * height)
                    box_width = (xmax - xmin) / width
                    box_height = (ymax - ymin) / height

                    yolo_file.write(
                        f"{class_id} {x_center} {y_center} {box_width} {box_height}\n")


voc_folder = "NumberPlateDataset_1/annotations"
yolo_folder = "NumberPlateDataset_1_yolo/labels"
classes = ["licence"]

convert_voc_to_yolo(voc_folder, yolo_folder, classes)

## Visualizing dataset


In [ ]:
import cv2
import os


def draw_bounding_box(image, label_path, class_names, normalized=True):
    with open(label_path, 'r') as label_file:
        lines = label_file.readlines()

    for line in lines:
        class_id, x_center, y_center, box_width, box_height = map(
            float, line.split())
        class_id = int(class_id)

        if not normalized:
            # If coordinates are not normalized, convert them to normalized
            width, height = image.shape[1], image.shape[0]
            x_center /= width
            y_center /= height
            box_width /= width
            box_height /= height

        # Convert YOLO format to (xmin, ymin, xmax, ymax)
        width, height = image.shape[1], image.shape[0]
        xmin = int((x_center - box_width / 2) * width)
        ymin = int((y_center - box_height / 2) * height)
        xmax = int((x_center + box_width / 2) * width)
        ymax = int((y_center + box_height / 2) * height)

        # Draw bounding box
        color = (0, 255, 0)  # Green color
        thickness = 2
        image = cv2.rectangle(image, (xmin, ymin),
                              (xmax, ymax), color, thickness)

        # Draw class name
        class_name = class_names[class_id]
        text = f"{class_name}"
        org = (xmin, ymin - 5)  # Slightly above the bounding box
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        cv2.putText(image, text, org, font, font_scale,
                    color, thickness, cv2.LINE_AA)

    return image


def save_images_with_annotations(image_folder, label_folder, output_folder, class_names, normalized=True):
    for filename in os.listdir(image_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(image_folder, filename)
            label_path = os.path.join(
                label_folder, os.path.splitext(filename)[0] + ".txt")

            if os.path.exists(label_path):
                image = cv2.imread(image_path)
                image_with_boxes = draw_bounding_box(
                    image, label_path, class_names, normalized)

                output_path = os.path.join(output_folder, filename)
                cv2.imwrite(output_path, image_with_boxes)


image_folder = "NumberPlateDataset_1_yolo/images"
label_folder = "NumberPlateDataset_1_yolo/labels"
output_folder = "Output/Sample_Images"
class_names = ["licence"]  # Update with your class names

save_images_with_annotations(
    image_folder, label_folder, output_folder, class_names)

In [7]:
import torch
torch.cuda.empty_cache()
%cd yolov5
!python train.py --data ../trainconfig.yaml --weights yolov5x.pt --img 640 --batch 1 --epochs 20 --project=../NumberPlateDataset_1_yolo --name=run_1 --exist-ok

[Errno 2] No such file or directory: 'yolov5'
/media/quantum410/windows-mount/projects/ANPR/yolov5_implementation/yolov5
train: weights=yolov5x.pt, cfg=, data=../trainconfig.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=1, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=../NumberPlateDataset_1_yolo, name=run_1, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: ⚠️ YOLOv5 is out of date by 4 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-278-g050c72cb Python-3.10.12 torch-2

In [10]:
%cd ..
!python yolov5/val.py --weights NumberPlateDataset_1_yolo/run_1/weights/best.pt --data trainconfig.yaml --img 640 --half --conf-thres 0.5 --project  Output/Detected

/media/quantum410/windows-mount/projects/ANPR/yolov5_implementation


/media/quantum410/windows-mount/projects/ANPR/yolov5_implementation/yolov5-venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


val: data=trainconfig.yaml, weights=['NumberPlateDataset_1_yolo/run_1/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.5, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=Output/Detected, name=exp, exist_ok=False, half=True, dnn=False
WARNING ⚠️ confidence threshold 0.5 > 0.001 produces invalid results
YOLOv5 🚀 v7.0-278-g050c72cb Python-3.10.12 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1050, 3010MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
val: Scanning /media/quantum410/windows-mount/projects/ANPR/yolov5_implementatio
                 Class     Images  Instances          P          R      mAP50   
                   all        112        120      0.954      0.933      0.954       0.62
Speed: 0.4ms pre-process, 115.0ms inference, 12.7ms NMS per image at shape (32, 3, 640, 640)
Results saved

In [11]:
!python yolov5/detect.py --weights  NumberPlateDataset_1_yolo/run_1/weights/best.pt --source assets/traffic-1.mp4 --conf 0.25 --project Output/Detected_Videos --name detect_video --exist-ok

detect: weights=['NumberPlateDataset_1_yolo/run_1/weights/best.pt'], source=assets/traffic-1.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=Output/Detected_Videos, name=detect_video, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-278-g050c72cb Python-3.10.12 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1050, 3010MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
video 1/1 (1/307) /media/quantum410/windows-mount/projects/ANPR/yolov5_implementation/assets/traffic-1.mp4: 384x640 3 LicencePlates, 116.8ms
video 1/1 (2/307) /media/quantum410/windows-mount/projects/ANPR/yolov5_implementation/assets/traffic-1.mp4: 384x640 3 LicencePla

In [12]:
!python yolov5/detect.py --weights  NumberPlateDataset_1_yolo/run_1/weights/best.pt --source assets/traffic-2.mp4 --conf 0.25 --project Output/Detected_Videos --name detect_video_2 --exist-ok

detect: weights=['NumberPlateDataset_1_yolo/run_1/weights/best.pt'], source=assets/traffic-2.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=Output/Detected_Videos, name=detect_video_2, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-278-g050c72cb Python-3.10.12 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1050, 3010MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
video 1/1 (1/303) /media/quantum410/windows-mount/projects/ANPR/yolov5_implementation/assets/traffic-2.mp4: 384x640 3 LicencePlates, 127.9ms
video 1/1 (2/303) /media/quantum410/windows-mount/projects/ANPR/yolov5_implementatio

In [13]:
!python yolov5/detect.py --weights  NumberPlateDataset_1_yolo/run_1/weights/best.pt --source assets/traffic-3.mp4 --conf 0.25 --project Output/Detected_Videos --name detect_video_3 --exist-ok

detect: weights=['NumberPlateDataset_1_yolo/run_1/weights/best.pt'], source=assets/traffic-3.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=Output/Detected_Videos, name=detect_video_3, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-278-g050c72cb Python-3.10.12 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1050, 3010MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
video 1/1 (1/625) /media/quantum410/windows-mount/projects/ANPR/yolov5_implementation/assets/traffic-3.mp4: 384x640 (no detections), 115.9ms
video 1/1 (2/625) /media/quantum410/windows-mount/projects/ANPR/yolov5_implementation/assets/traffic-3.mp4: 384x640 (no detect

### New Dataset

In [4]:
!python yolov5/val.py --weights NumberPlateDataset_1_yolo/run_1/weights/best.pt --data trainconfig.yaml --img 640 --half --conf-thres 0.25 --project  Output/New_Detected

val: data=trainconfig.yaml, weights=['NumberPlateDataset_1_yolo/run_1/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.25, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=Output/New_Detected, name=exp, exist_ok=False, half=True, dnn=False
WARNING ⚠️ confidence threshold 0.25 > 0.001 produces invalid results
YOLOv5 🚀 v7.0-278-g050c72cb Python-3.10.12 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1050, 3010MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
val: Scanning /media/quantum410/windows-mount/projects/ANPR/yolov5_implementatio
                 Class     Images  Instances          P          R      mAP50   
                   all        663        859          0          0          0          0
Speed: 0.5ms pre-process, 158.9ms inference, 1.1ms NMS per image at shape (32, 3, 640, 640)
Results 

In [6]:
!python yolov5/detect.py --weights  NumberPlateDataset_1_yolo/run_1/weights/best.pt --source LicensePlateDataset/val/images --conf 0.25 --project Output/New_val_detections --exist-ok

detect: weights=['NumberPlateDataset_1_yolo/run_1/weights/best.pt'], source=LicensePlateDataset/val/images, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=Output/New_val_detections, name=exp, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-278-g050c72cb Python-3.10.12 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1050, 3010MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
image 1/663 /media/quantum410/windows-mount/projects/ANPR/yolov5_implementation/LicensePlateDataset/val/images/frame1001_jpg.rf.0501f7e1ac37736e79f8ae8aed78e414.jpg: 640x640 (no detections), 178.0ms
image 2/663 /media/quantum410/windows-mount/projects/ANPR/yolov5_i

- None of the images got a single bounding box detected shows the less diversity of the dataset.


In [8]:
import torch
torch.cuda.empty_cache()
%cd yolov5
!python train.py --data ../trainconfig.yaml --weights ../NumberPlateDataset_1_yolo/run_1/weights/best.pt --img 640 --batch 1 --epochs 20 --project=../LicensePlateDataset --name=run_2 --exist-ok

[Errno 2] No such file or directory: 'yolov5'
/media/quantum410/windows-mount/projects/ANPR/yolov5_implementation/yolov5


/media/quantum410/windows-mount/projects/ANPR/yolov5_implementation/yolov5-venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


train: weights=../NumberPlateDataset_1_yolo/run_1/weights/best.pt, cfg=, data=../trainconfig.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=1, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=../LicensePlateDataset, name=run_2, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: ⚠️ YOLOv5 is out of date by 4 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-278-g050c72cb Python-3.10.12 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1050, 3010MiB)

hyperparameters: lr0=0.01, lrf=0.